In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
#imports libraries 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import LancasterStemmer, PorterStemmer
from nltk.stem import SnowballStemmer
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


In [2]:
file = '/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/train_data.txt'

In [152]:
# Load the dataset using pandas with custom delimiter
train = pd.read_csv(file, delimiter=' ::: ', engine='python', header=None, names=['ID', 'Title', 'Genre', 'Description'])

In [92]:
train

,ID,Title,Genre,Description
0,1,Oscar et la dame rose (2009),drama,Listening in to a conversation between his doc...
1,2,Cupid (1997),thriller,A brother and sister with a past incestuous re...
2,3,"Young, Wild and Wonderful (1980)",adult,As the bus empties the students for their fiel...
3,4,The Secret Sin (1915),drama,To help their unemployed father make ends meet...
4,5,The Unrecovered (2007),drama,The film's title refers not only to the un-rec...
...,...,...,...,...
54209,54210,"""Bonino"" (1953)",comedy,This short-lived NBC live sitcom centered on B...
54210,54211,Dead Girls Don't Cry (????),horror,The NEXT Generation of EXPLOITATION. The siste...
54211,54212,Ronald Goedemondt: Ze bestaan echt (2008),documentary,"Ze bestaan echt, is a stand-up comedy about gr..."
54212,54213,Make Your Own Bed (1944),comedy,Walter and Vivian live in the country and have...


In [5]:
train.shape

(54214, 4)

In [6]:
file2 = "/kaggle/input/genre-classification-dataset-imdb/Genre Classification Dataset/test_data.txt"

In [151]:
# Load test dataset with a custom delimiter and column names using pandas
test = pd.read_csv(file2, delimiter=' ::: ', engine='python', header=None, names=['ID', 'Title',  'Description'])

In [94]:
test.head(10)

,ID,Title,Description
0,1,Edgar's Lunch (1998),"L.R. Brane loves his life - his car, his apart..."
1,2,La guerra de papá (1977),"Spain, March 1964: Quico is a very naughty chi..."
2,3,Off the Beaten Track (2010),One year in the life of Albin and his family o...
3,4,Meu Amigo Hindu (2015),"His father has died, he hasn't spoken with his..."
4,5,Er nu zhai (1955),Before he was known internationally as a marti...
5,6,Riddle Room (2016),Emily Burns is being held captive in a room wi...
6,7,L'amica (1969),The beautiful but neglected wife of a brillian...
7,8,Ina Mina Dika (1989),Vasu Inamdar (Ina) suffers from a disorder whe...
8,9,Equinox Special: Britain's Tornados (2005),An insight into the tornados that hit Kensal R...
9,10,Press (2011),Press is a story of young people overwhelmed b...


In [95]:
train.columns

Index(['ID', 'Title', 'Genre', 'Description'], dtype='object')

In [96]:
test.columns

Index(['ID', 'Title', 'Description'], dtype='object')

In [97]:
# Add a 'Genre' column with None values to the test dataset
test['Genre'] = None

In [98]:
test.head(10)

,ID,Title,Description,Genre
0,1,Edgar's Lunch (1998),"L.R. Brane loves his life - his car, his apart...",None
1,2,La guerra de papá (1977),"Spain, March 1964: Quico is a very naughty chi...",None
2,3,Off the Beaten Track (2010),One year in the life of Albin and his family o...,None
3,4,Meu Amigo Hindu (2015),"His father has died, he hasn't spoken with his...",None
4,5,Er nu zhai (1955),Before he was known internationally as a marti...,None
5,6,Riddle Room (2016),Emily Burns is being held captive in a room wi...,None
6,7,L'amica (1969),The beautiful but neglected wife of a brillian...,None
7,8,Ina Mina Dika (1989),Vasu Inamdar (Ina) suffers from a disorder whe...,None
8,9,Equinox Special: Britain's Tornados (2005),An insight into the tornados that hit Kensal R...,None
9,10,Press (2011),Press is a story of young people overwhelmed b...,None


In [12]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54214 entries, 0 to 54213
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ID           54214 non-null  int64 
 1   Title        54214 non-null  object
 2   Genre        54214 non-null  object
 3   Description  54214 non-null  object
dtypes: int64(1), object(3)
memory usage: 1.7+ MB


In [13]:
train.describe()

,ID
count,54214.000000
mean,27107.500000
std,15650.378084
min,1.000000
25%,13554.250000
50%,27107.500000
75%,40660.750000
max,54214.000000


In [14]:
train.isnull().sum()

ID             0
Title          0
Genre          0
Description    0
dtype: int64

In [103]:
test.isnull().sum()

ID                 0
Title              0
Description        0
Genre          54200
dtype: int64

In [12]:
train.duplicated().sum()

0

In [17]:
train.size

216856

In [18]:
train.size

216856

In [21]:
train.count()

ID             54214
Title          54214
Genre          54214
Description    54214
dtype: int64

In [99]:
# Count the number of non-null entries in each column of the test dataset
test.count()

ID             54200
Title          54200
Description    54200
Genre              0
dtype: int64

In [153]:
# Encode the 'Genre' column in the train dataset using LabelEncoder
encoding = LabelEncoder()
train['Genre'] = encoding.fit_transform(train['Genre'].values)

In [154]:
# Count the occurrences of each genre in the train dataset
train['Genre'].value_counts()

Genre
8     13613
7     13096
5      7447
21     5073
13     2204
24     1591
0      1315
26     1032
18      884
9       784
2       775
14      731
19      672
20      647
1       590
6       505
3       498
22      432
23      391
10      323
16      319
15      277
4       265
12      243
11      194
17      181
25      132
Name: count, dtype: int64

In [155]:
# Concatenate train and test datasets vertically into a single DataFrame
df = pd.concat([train,test],axis=0)

In [156]:
df.head()

,ID,Title,Genre,Description
0,1,Oscar et la dame rose (2009),8.0,Listening in to a conversation between his doc...
1,2,Cupid (1997),24.0,A brother and sister with a past incestuous re...
2,3,"Young, Wild and Wonderful (1980)",1.0,As the bus empties the students for their fiel...
3,4,The Secret Sin (1915),8.0,To help their unemployed father make ends meet...
4,5,The Unrecovered (2007),8.0,The film's title refers not only to the un-rec...


In [157]:
df.isnull().sum()

ID                 0
Title              0
Genre          54200
Description        0
dtype: int64

In [158]:
# Fill missing values in the 'Genre' column with the mean value (if applicable)
df['Genre'] = df['Genre'].fillna(df['Genre'].mean())

In [159]:
# Convert the 'Genre' column to integer data type
df['Genre'] = df['Genre'].astype("int")

In [160]:
df['Genre'].value_counts()

Genre
10    54523
8     13613
7     13096
5      7447
21     5073
13     2204
24     1591
0      1315
26     1032
18      884
9       784
2       775
14      731
19      672
20      647
1       590
6       505
3       498
22      432
23      391
16      319
15      277
4       265
12      243
11      194
17      181
25      132
Name: count, dtype: int64

In [161]:
# Drop the 'Title' column from the DataFrame
df.drop('Title', inplace = True, axis=1)
# Drop the 'ID' column from the DataFrame
df.drop('ID', inplace = True, axis=1)

In [163]:
# Check for any null values in the DataFrame
df.isnull().any()

Genre          False
Description    False
dtype: bool

In [164]:
# Convert all text in the 'Description' column to lowercase
df['Description'].str.lower()

0        listening in to a conversation between his doc...
1        a brother and sister with a past incestuous re...
2        as the bus empties the students for their fiel...
3        to help their unemployed father make ends meet...
4        the film's title refers not only to the un-rec...
                               ...                        
54195    covering multiple genres, tales of light & dar...
54196    as alice and cora munro attempt to find their ...
54197    a movie 169 years in the making. oliver twist,...
54198    popular, but mysterious rock d.j mike mallard ...
54199    curitiba is a city in movement, with rhythms a...
Name: Description, Length: 108414, dtype: object

In [165]:
# Define a function to remove URLs from the input text

def removeurl(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r' ',text)

In [166]:
# Apply the removeurl function to the 'Description' column to remove URLs
df['Description']=df['Description'].apply(removeurl)

In [167]:
# Define a function to remove HTML tags from the input text

def removehtml(text):
    html = re.compile(r'<.*?>')
    return html.sub(r' ',text)
    

In [168]:
# Apply the removehtml function to the 'Description' column to remove HTML tags
df['Description']=df['Description'].apply(removehtml)

In [169]:
# Define a function to remove punctuation from the input text
def removepunctuation(text):
    punc = string.punctuation
    return text.translate(str.maketrans('','',punc))

In [170]:
df['Description']=df['Description'].apply(removepunctuation)

In [171]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [173]:
# Define a function to remove stopwords from the input text
def remove_stopword(text):
    stopwords = set(stopwords.words("english"))
    return " ".join([word for word in text.split() if word.lower() not in stopwords])
    
            
            

In [123]:
df['Description'] = df['Description'].apply(remove_stopword)

In [124]:
df['Description'][4]

4    films title refers unrecovered bodies ground z...
4    known internationally martial arts superstar B...
Name: Description, dtype: object

In [174]:
# Initialize the Snowball Stemmer for English language
snowball_stemmer=SnowballStemmer('english')

In [175]:
def stemming(text):
    # Tokenize the input text
    tokens = word_tokenize(text)
    # Apply stemming to each token
    stemmed_tokens = [snowball_stemmer.stem(word) for word in tokens]
    # Join the stemmed tokens back into a single string
    return " ".join(stemmed_tokens)

In [176]:
# Apply the stemming function to the 'Description' column to normalize words
df['Description'] = df['Description'].apply(stemming)

In [177]:
df['Description'][4]

4    the film titl refer not onli to the unrecov bo...
4    befor he was known intern as a martial art sup...
Name: Description, dtype: object

In [178]:
df['Description'].str.lower()

0        listen in to a convers between his doctor and ...
1        a brother and sister with a past incestu relat...
2        as the bus empti the student for their field t...
3        to help their unemploy father make end meet ed...
4        the film titl refer not onli to the unrecov bo...
                               ...                        
54195    cover multipl genr tale of light dark is an an...
54196    as alic and cora munro attempt to find their f...
54197    a movi 169 year in the make oliv twist the art...
54198    popular but mysteri rock dj mike mallard askew...
54199    curitiba is a citi in movement with rhythm and...
Name: Description, Length: 108414, dtype: object

In [179]:
df.columns

Index(['Genre', 'Description'], dtype='object')

In [180]:
df.head(10)

,Genre,Description
0,8,listen in to a convers between his doctor and ...
1,24,a brother and sister with a past incestu relat...
2,1,as the bus empti the student for their field t...
3,8,to help their unemploy father make end meet ed...
4,8,the film titl refer not onli to the unrecov bo...
5,7,qualiti control consist of a seri of 16mm sing...
6,5,in tough econom time max and joey have all but...
7,6,ron petri keanu reev is a troubl teen whose li...
8,18,a sudden calamit event caus great loss of life...
9,13,four high school student embark on a terrifi j...


In [181]:
df['Genre'].value_counts()

Genre
10    54523
8     13613
7     13096
5      7447
21     5073
13     2204
24     1591
0      1315
26     1032
18      884
9       784
2       775
14      731
19      672
20      647
1       590
6       505
3       498
22      432
23      391
16      319
15      277
4       265
12      243
11      194
17      181
25      132
Name: count, dtype: int64

In [183]:
# Define feature and target variables: X for descriptions and y for genres
X = df['Description']
y = df['Genre']

In [184]:
# Initialize TF-IDF vectorizer and transform the 'Description' text data into numerical features
vectorizer = TfidfVectorizer()  
X = vectorizer.fit_transform(X)


In [185]:


X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25,random_state=42)

In [186]:
X_train.shape

(81310, 177974)

In [187]:
y_train.shape

(81310,)

In [94]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)

In [95]:
## X_train and y_train after resampling

X_train, y_train = smote.fit_resample(X_train,y_train)

In [96]:
print(f"Resampled training data shape: {X_train.shape}, {y_train.shape}")

Resampled training data shape: (276453, 114943), (276453,)


In [188]:
y_train.value_counts()

Genre
10    41008
8     10123
7      9788
5      5571
21     3836
13     1651
24     1205
0       993
26      774
18      641
9       593
2       579
14      551
19      509
20      493
1       445
3       372
6       363
22      321
23      281
16      249
15      210
4       193
12      165
11      151
17      138
25      107
Name: count, dtype: int64

In [98]:
from sklearn.svm import SVC

In [119]:
svc = SVC()

In [120]:
svc.fit(X_train,y_train)

SVC()

In [121]:
predict = svc.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test,predict)
accuracy  = accuracy*100
print(f"accuracy is", accuracy,"%")

In [189]:
from sklearn.naive_bayes import MultinomialNB

In [190]:
NB = MultinomialNB()

In [191]:
NB.fit(X_train,y_train)

MultinomialNB()

In [192]:
prediction = NB.predict(X_test)

In [193]:
accuracy = accuracy_score(y_test,prediction)
accuracy  = accuracy*100
print(f"accuracy is", accuracy,"%")

accuracy is 49.79707792207792 %


In [201]:
from sklearn.metrics import confusion_matrix

In [202]:
print(confusion_matrix(y_test,prediction))

[[    0     0     0     0     0     0     0     0     0     0   322     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0]
 [    0     0     0     0     0     0     0     0     0     0   145     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0]
 [    0     0     0     0     0     2     0     0     0     0   194     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0]
 [    0     0     0     0     0     0     0     1     0     0   125     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0]
 [    0     0     0     0     0     0     0     0     0     0    72     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0]
 [    0     0     0     0     0    15     0     0     3     0  1858     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0

In [205]:
from sklearn.metrics import classification_report

In [206]:
print(classification_report(y_test,prediction))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       322
           1       0.00      0.00      0.00       145
           2       0.00      0.00      0.00       196
           3       0.00      0.00      0.00       126
           4       0.00      0.00      0.00        72
           5       0.28      0.01      0.02      1876
           6       0.00      0.00      0.00       142
           7       0.67      0.00      0.00      3308
           8       0.22      0.00      0.00      3490
           9       0.00      0.00      0.00       191
          10       0.50      1.00      0.66     13515
          11       0.00      0.00      0.00        43
          12       0.00      0.00      0.00        78
          13       0.00      0.00      0.00       553
          14       0.00      0.00      0.00       180
          15       0.00      0.00      0.00        67
          16       0.00      0.00      0.00        70
          17       0.00    

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [194]:
from sklearn.linear_model import LogisticRegression

In [195]:
lr = LogisticRegression()

In [196]:
lr.fit(X_train,y_train)

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [197]:
predict = lr.predict(X_test)

In [198]:
accuracy = accuracy_score(y_test,predict)
accuracy  = accuracy*100
print(f"accuracy is", accuracy,"%")

accuracy is 48.561097992916174 %
